<a href="https://colab.research.google.com/github/Utterbackian/Neuromatch2023_Medical_Imaging/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import shutil

In [140]:
path = "../eeg_5_95_std.pth"
loaded_object_5_95 = torch.load(path) 

# Uncomment for google colab 

# from google.colab import drive
# drive.mount('/content/drive')

# file_path = '/content/drive/MyDrive/data/eeg_5_95_std.pth'
# loaded_object_5_95 = torch.load(file_path)

## Indexing

In [141]:
def indexing(field):
    idx_dict = {}
    for i, image in enumerate(loaded_object_5_95[field]):
        idx_dict[i] = image
    return idx_dict

In [142]:
for EEG in loaded_object_5_95['dataset']:
    indexed_labels = indexing("labels")
    indexed_images = indexing("images")
    EEG['image'] = indexed_images[EEG['image']]
    EEG['label'] = indexed_labels[EEG['label']]

## LSTM Encoder

In [143]:
import torch
import torch.nn as nn

class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bidirectional=False):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, 128)  # Output 128 units for 1D output

    def forward(self, x):
        # x: Input sequence of shape (batch_size, seq_length, input_size)
        # Output shape: (batch_size, 128)

        # Pass the input sequence through the LSTM layer
        lstm_output, _ = self.lstm(x)

        # Use AdaptiveAvgPool1d to handle variable sequence lengths
        lstm_output = nn.AdaptiveAvgPool1d(1)(lstm_output.permute(0, 2, 1)).squeeze(2)

        # Pass through the fully connected layer for 1D output
        output = self.fc(lstm_output)

        return output

In [144]:
input_size = 491
hidden_size = 128  # Dimension of the hidden states in the LSTM
num_layers = 2  # Number of LSTM layers
bidirectional = True  # Whether to use bidirectional LSTMs or not
batch_size = 5
eeg_data = loaded_object_5_95['dataset'][0]['eeg'] # one eeg sequence

## Add Noise

In [145]:
def addNoise(output, sigma=0.1):
    # Generate Gaussian noise with the same shape as the encoded vector
    noise = torch.randn_like(output) * sigma

    # Add the noise to the encoded vector
    return torch.cat((output, noise), axis=1)

In [146]:
torch.manual_seed(2023)

In [147]:
# makes all eeg 128, 500

# CHANGE IT TO 491 
for i in loaded_object_5_95['dataset']:
    i['eeg'].resize_(128, 491)

In [148]:
# CURRENTLY THIS CODE CAUSES VSCODE TO CRASH
#                  |
#                  |
#                  V

def process_eeg(eeg_data):
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    eeg_data = eeg_data.view(1, hidden_size, input_size)#.to(device)
    eeg_data = encoder(eeg_data)
    return addNoise(eeg_data)

In [149]:
encoder = LSTMEncoder(491, hidden_size, num_layers, bidirectional)

encoded_data = []
for i in loaded_object_5_95['dataset'][:500]:
    encoded_data.append([i['image'], process_eeg(i['eeg'])])

## Dataset

In [151]:
import glob
import cv2
from torch.utils.data import Dataset, DataLoader

In [152]:
class CustomDataset(Dataset):
    def __init__(self):
        
        # Uncomment for local
        # self.imgs_path = "../filtered/"

        # # Uncomment for google colab
        # # self.imgs_path = '/content/drive/MyDrive/filtered/'

        # file_list = glob.glob(self.imgs_path + "*")

        # print(file_list)
        self.data = []
        for eeg in encoded_data:
            class_path = "../filtered/" + eeg[0] + ".JPEG"
            self.data.append([class_path, eeg[1]])
        # for class_path in file_list:
        #     class_name = class_path.split("\\")[-1]
        #     self.data.append([class_path, class_name.split()[0]])
        self.img_dim = (64, 64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx][0]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        img_tensor = torch.from_numpy(img)
        return img_tensor.float().permute(2,0,1), self.data[idx][-1]
    
# dataset = CustomDataset()
# dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

## Model

In [153]:
import torch
import torch.nn as nn


class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            # nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            # nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


def test():
    N, in_channels, H, W = 8, 3, 64, 64
    noise_dim = 256
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
    print("Success, tests passed!")


if __name__ == "__main__":
    test()

Success, tests passed!


In [154]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 1
IMAGE_SIZE = 64
CHANNELS_IMG = 3
NOISE_DIM = 256
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64

transforms = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)

# If you train on MNIST, remember to set channels_img to 1

dataset = CustomDataset()

# comment mnist above and uncomment below if train on CelebA
# dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, eeg) in enumerate(dataloader):
        real = real.to(device)
        noise = eeg.reshape(1, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward(retain_graph=True)
        opt_gen.step()

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

Epoch [0/5] Batch 0/500                   Loss D: 1.9180, loss G: 0.6302
Epoch [0/5] Batch 100/500                   Loss D: 0.0033, loss G: 5.0939
Epoch [0/5] Batch 200/500                   Loss D: 0.0008, loss G: 6.4343
Epoch [0/5] Batch 300/500                   Loss D: 0.0003, loss G: 7.6079
Epoch [0/5] Batch 400/500                   Loss D: 0.0001, loss G: 8.2385
Epoch [1/5] Batch 0/500                   Loss D: 0.0001, loss G: 8.7205
Epoch [1/5] Batch 100/500                   Loss D: 0.0001, loss G: 9.1320
Epoch [1/5] Batch 200/500                   Loss D: 0.0000, loss G: 9.4491
Epoch [1/5] Batch 300/500                   Loss D: 0.0000, loss G: 9.7644
Epoch [1/5] Batch 400/500                   Loss D: 50.0000, loss G: 0.0000
Epoch [2/5] Batch 0/500                   Loss D: 50.0000, loss G: 0.0000
Epoch [2/5] Batch 100/500                   Loss D: 50.0000, loss G: 0.0000
Epoch [2/5] Batch 200/500                   Loss D: 50.0000, loss G: 0.0000
Epoch [2/5] Batch 300/500  